# Rosenblatt-Lernregel

## Einlesen und Vorbereiten der Daten

In [ ]:
# Lege fest, ob das Notebook in der lite-Version asugeführt wird
# (Bei lokaler Ausführung müssen einige Zeilen geringfügig angepasst werden)
run_in_lite_version = True

In [ ]:
# nur für jupyter lite notwendig
if run_in_lite_version:
    %pip install -q plotly 
    from IPython.display import display, HTML

In [ ]:
# Importiere benötigte Pakete
import pandas as pd
import plotly.express as px

In [ ]:
# Importiere Daten
df_tiere = pd.read_csv('tiere_gefaehrlichkeit.csv', sep=';')
df_tiere.head()

In [ ]:
# Visualisiere Daten
fig = px.scatter(df_tiere, x="Zahngröße", y="Augengröße",color="Gefahr",height=350, width=400)
if run_in_lite_version:
    display(HTML(fig.to_html())) # jupyter lite
else:
    fig.show() # jupyter notebook lokal

In [ ]:
# Erzeuge numerische Daten
df_tiere_num = df_tiere.replace(['ungefährlich','gefährlich'],[0,1])
df_tiere_num.head()

## Perzeptron

In [ ]:
# Definiere Gewichte und Schwellwert als globale Variablen
w1 = 1
w2 = 1
theta = 1

In [ ]:
#Funktion für den Output des KNN 
def perceptron_output(x1, x2): 
    if  w1*x1+w2*x2>= theta:
        return 1
    else:
        return 0

In [ ]:
# Teste Perzeptron
perceptron_output(0,1)

## Heatmap

In [ ]:
def plot_heatmap(x_min=0, x_max=4, y_min=0, y_max=3,resolution = 50):
    delta_x = x_max-x_min
    delta_y = y_max-y_min

    heatmap = pd.DataFrame(index = [y_min+i*delta_y/resolution \
                                    for i in reversed(range(resolution+1))])

    for x in [x_min+i*delta_x/resolution for i in range(resolution+1)]:
        heatmap[x] = [perceptron_output(x,y_min+i*delta_y/resolution) \
                      for i in reversed(range(resolution+1))]
    
    #display(heatmap)
    fig = px.imshow(heatmap, text_auto=False, color_continuous_scale="RdBu_r", \
                    height=300, width=300,\
                    origin='lower',zmin=0,zmax=1,\
                        labels=dict(x="Zahngröße", y="Augengröße", color="Gefahr"))
    
    if run_in_lite_version:
        display(HTML(fig.to_html())) # jupyter lite
    else:
        fig.show() # jupyter notebook lokal

In [ ]:
plot_heatmap()

## Funktion zum Testen der Vorhersage

In [ ]:
# Funktion zur Berechnung der Genauigkeit des Perzeptrons
# Die Funktion gibt den Anteil der korrekt klassifizierten Tiere zurück
def accuracy():
    #Erstelle Spalte mit interpretierten Outputs des KNN
    Outputs=[]

    for i in range(len(df_tiere)):
        output = perceptron_output(df_tiere.iloc[i]['Zahngröße'],df_tiere.iloc[i]['Augengröße'])
        
        if output > 0:
            Outputs.append('gefährlich')
        elif output <= 0:
            Outputs.append('ungefährlich')
            
    df_tiere['Vorhersagen'] = Outputs

    #display(df_tiere.head() )
    return sum(df_tiere['Gefahr'] == df_tiere['Vorhersagen']) / len(df_tiere)

In [ ]:
accuracy()

In [ ]:
# Falsch Klassifizierte Tiere
df_tiere[df_tiere['Gefahr'] != df_tiere['Vorhersagen']]

In [ ]:
# Berechne wie viele Beispiele korrekt klassifiziert werden 
sum(df_tiere['Gefahr'] == df_tiere['Vorhersagen'])

## Lernprozess (Delta-Regel)

In [ ]:
# Delta-Regel, alle Datenssätzen, Minibatch der Größe Eins
def delta_regel(w1, w2, theta):
    for i in range(len(df_tiere_num)):
        Y = df_tiere_num.iloc[i]['Gefahr'] # Klassifikation des Tiers
        y = perceptron_output(df_tiere_num.iloc[i]['Zahngröße'],df_tiere_num.iloc[i]['Augengröße'])
        delta = Y-y # Outputfehler des Perzeptrons
        w1    += lr*delta*df_tiere_num.iloc[i]['Zahngröße'] # lr: Lernrate
        w2    += lr*delta*df_tiere_num.iloc[i]['Augengröße']
        theta -= lr*delta 
    return w1, w2, theta

## Einzelne Lern-Epoche

In [ ]:
# Lernrate
lr=0.3

In [ ]:
# Parameter für das Perzeptron
w1 = 1 #-1
w2 = 0.5 #1
theta = 1

In [ ]:
print("Anteil korrekte Vorhersagen vor dem Training:  ", accuracy())
w1, w2, theta = delta_regel(w1, w2, theta)
print("Anteil korrekte Vorhersagen nach dem Training: ", accuracy())
plot_heatmap()

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 1:
Führe die Delta-Regel für das Perzeptron für 10 Iterationen durch. Wie verändert sich die Genauigkeit des Perzeptrons? Halte deine Ergebnisse schriftlich fest.


Ergebnisse bitte hierher

## Mehrere Lern-Epochen auf einmal

In [ ]:
# Berechne n Epochen, Abbbruch wenn 100% erreicht
def calc_epochs_delta(n):
    global w1, w2, theta
    for epoch in range(n):
        print(" Calc epoch no.", epoch, end=" ... ")
        w1, w2, theta = delta_regel(w1, w2, theta)
        #print(w1,w2,theta, end=" ")
        print("Accuarcy: ", accuracy())
        if accuracy() == 1:
            return epoch

In [ ]:
# Lernrate
lr=1

In [ ]:
#Festlegung der Parameter für das Perzeptron
w1    =  1
w2    = -1 
theta =  1 

In [ ]:
# Anzahl der Epochen bis 100% Genauigkeit erreicht sind
print( "Number of epochs to 100% :", calc_epochs_delta(100) )

In [ ]:
# Zeichne Heatmap und Vergleich mit den Daten
plot_heatmap(x_min=0, x_max=4, y_min=0, y_max=4,resolution = 100)
fig = px.scatter(df_tiere, x="Zahngröße", y="Augengröße",color="Gefahr",\
                 height=280, width=350)

if run_in_lite_version:
    display(HTML(fig.to_html())) # jupyter lite
else:
    fig.show() # jupyter notebook lokal

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 2:
Teste die Konvergenzrate für unterschiedliche Lernraten und Parameter des Perzeptrons. Findest du eine Lernrate bzw. Paramter des Perzeptrons, für die das Verfahren nicht konvergiert?


Ergebnisse bitte hierher

## Fertiges Modell
Im fertigen Modell werden die Gewichte und der Schwellenwert zufällig vorbelegt.

In [ ]:
import random

In [ ]:
lr = 0.01
w1 = random.normalvariate(-10,3)
w2 = random.normalvariate(10,5)
theta = random.normalvariate(3,3)

In [ ]:
print( "Number of epochs to 100% :", calc_epochs_delta(100) )

<div style="padding: 5px; border: 5px solid #0077b6;">

### Aufgabe 3:
Teste das fertige Modell für unterschiedliche Lernraten und unterschiedliche zufällige Vorbelegung der Parameter des Perzeptrons. Dazu musst du recherchieren, was die Funktion `random.normalvariate` macht. Halte deine Beobachtungen schriftlich fest.

Ergebnisse bitte hierher